In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd /content/drive/MyDrive/assessmentanalysis/

/content/drive/MyDrive/assessmentanalysis


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models, datasets
import os
from PIL import Image
from sklearn.model_selection import train_test_split
import shutil
import numpy as np

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def create_train_val_folders(data_dir, val_size=0.2):
    classes = os.listdir(data_dir)
    os.makedirs(os.path.join(data_dir, 'train'), exist_ok=True)
    os.makedirs(os.path.join(data_dir, 'val'), exist_ok=True)

    for cls in classes:
        os.makedirs(os.path.join(data_dir, 'train', cls), exist_ok=True)
        os.makedirs(os.path.join(data_dir, 'val', cls), exist_ok=True)
        # Get all the images
        images = os.listdir(os.path.join(data_dir, cls))
        # Split the images
        train_imgs, val_imgs = train_test_split(images, test_size=val_size, random_state=42)
        # Move images to respective folders
        for img in train_imgs:
            shutil.move(os.path.join(data_dir, cls, img), os.path.join(data_dir, 'train', cls))
        for img in val_imgs:
            shutil.move(os.path.join(data_dir, cls, img), os.path.join(data_dir, 'val', cls))

# Call the function with the path of dataset
data_dir = './dataset'
create_train_val_folders(data_dir)


In [7]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Load data from folders
data_dir = './dataset'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


In [ ]:
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # Changing the final layer for binary classification (code or other)

model = model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 219MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) # Stochastic gradient descent


In [10]:
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    return model

model = train_model(model, criterion, optimizer, num_epochs=25)
torch.save(model.state_dict(), 'code_detect.pth')

Epoch 1/25
----------
train Loss: 0.7519 Acc: 0.4375
val Loss: 0.5125 Acc: 0.8333
Epoch 2/25
----------
train Loss: 0.4556 Acc: 0.7708
val Loss: 0.4540 Acc: 0.7500
Epoch 3/25
----------
train Loss: 0.5557 Acc: 0.7083
val Loss: 0.7466 Acc: 0.6667
Epoch 4/25
----------
train Loss: 0.5405 Acc: 0.7708
val Loss: 0.3798 Acc: 0.9167
Epoch 5/25
----------
train Loss: 0.7201 Acc: 0.7500
val Loss: 0.2412 Acc: 0.9167
Epoch 6/25
----------
train Loss: 0.3674 Acc: 0.8958
val Loss: 0.3281 Acc: 0.7500
Epoch 7/25
----------
train Loss: 0.3651 Acc: 0.8125
val Loss: 0.0723 Acc: 1.0000
Epoch 8/25
----------
train Loss: 0.8032 Acc: 0.6875
val Loss: 0.0950 Acc: 1.0000
Epoch 9/25
----------
train Loss: 0.5160 Acc: 0.7917
val Loss: 0.2599 Acc: 0.9167
Epoch 10/25
----------
train Loss: 0.5248 Acc: 0.7500
val Loss: 0.1471 Acc: 0.8333
Epoch 11/25
----------
train Loss: 0.7679 Acc: 0.7708
val Loss: 0.4075 Acc: 0.9167
Epoch 12/25
----------
train Loss: 0.2927 Acc: 0.8958
val Loss: 0.1154 Acc: 0.9167
Epoch 13/25
-

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image

data_transforms = {
   'val': transforms.Compose([
       transforms.Resize(256),
       transforms.CenterCrop(224),
       transforms.ToTensor(),
       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
   ])
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model structure
loaded_model = models.resnet18(pretrained=False)
num_ftrs = loaded_model.fc.in_features
loaded_model.fc = nn.Linear(num_ftrs, 2)

# Load the trained weights
loaded_model.load_state_dict(torch.load('code_detect.pth'))
loaded_model = loaded_model.to(device)
loaded_model.eval()  # Set to evaluation mode

def predict_image(image_path, model):
    """
    Predicts the class of an image using the provided model.

    Args:
    image_path (str): Path to the image file.
    model (torch.nn.Module): The trained model.

    Returns:
    str: The predicted class name.
    """
    image = Image.open(image_path).convert('RGB') # Convert image to RGB
    image = data_transforms['val'](image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)

    class_names = ['Code', 'Other']  # Define class names for classification
    return class_names[preds[0]]

# Predict class of an image by uing the function
image_path = '/content/drive/MyDrive/assessmentanalysis/dataset/train/Code/Screenshot (105).png'
print(predict_image(image_path, loaded_model))

Code


<ipython-input-19-25ae0be5f4d8>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load('code_detect.pth'))
